In [ ]:
from langchain_docling import DoclingLoader
from docling.chunking import HybridChunker

chunker = HybridChunker(max_tokens=4000)
loader = DoclingLoader('cv.pdf', chunker=chunker)

In [1]:
from llm_providers import GoogleAIModel
import os
from dotenv import load_dotenv
from multi_agent import SyntheticDataGenerator
from langchain_community.retrievers import BM25Retriever
from utils import pdf_parser
from tasks import Task

load_dotenv()
key = os.environ.get('GOOGLE_API_KEY')
llm = GoogleAIModel(api_key=key)
pdf_files = pdf_parser('cv.pdf')
batch_size = 3

bm25retriever = BM25Retriever.from_documents(pdf_files, k=batch_size)
thread_config = {'configurable': {'thread_id': 123}}

agent = SyntheticDataGenerator(llm=llm, 
                               retriever=bm25retriever,
                               output_path='./output',
                               buffer_size=5,
                               thread_id=thread_config)

example_task = Task(
    task_name="dpo",
    localization='Optimization, Data Science and AI',
    task_description="Generate a conversation between a HR manager and a candidate applying for a data science position using this information.",
    rows_per_batch=11,
    language="Vietnamese"
)

c:\Users\quock\Downloads\syndata_agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


╭───────────────────────────────────────────────────── ERROR ─────────────────────────────────────────────────────╮
│ Error parsing PDF:                                                                                              │
│                                                                                                                 │
│ Subscripted generics cannot be used with class and instance checks                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ValueError: Error parsing PDF file

In [ ]:
async for output in agent.agent_flow.astream(
    {
        "task": example_task,
        "human_feedback": None,
        "conversations": []
    }
    ,
    config=thread_config, 
    stream_mode="updates"):
    for key, value in output.items():
        print(f"{key}: {value}")
        if key == '__interrupt__':
            print("Ayo LIL homie")
    print("\n------------------------------\n")

In [ ]:
from langgraph.types import Command

await agent.agent_flow.ainvoke(
    Command(resume='no'),
    config=thread_config
)